In [16]:
pip install statsbombpy

Note: you may need to restart the kernel to use updated packages.


In [17]:
from statsbombpy import sb
import pandas as pd
from pandas import json_normalize
import numpy as np

In [18]:
matches = sb.matches(competition_id=11, season_id=27)
matches

d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3825848,2015-09-23,20:00:00.000,Spain - La Liga,2015/2016,Levante UD,Eibar,2,2,available,...,None,5,Regular Season,Estadio Ciudad de Valencia,NaN,Luis Lucas Alcaraz González,José Luis Mendilibar Etxebarria,1.1.0,2,2
1,3825895,2015-09-23,22:00:00.000,Spain - La Liga,2015/2016,Las Palmas,Sevilla,2,0,available,...,None,5,Regular Season,Estadio de Gran Canaria,Carlos del Cerro Grande,Francisco Herrera Lorenzo,Unai Emery Etxegoien,1.1.0,2,2
2,3825894,2016-05-01,18:15:00.000,Spain - La Liga,2015/2016,RC Deportivo La Coruña,Getafe,0,2,available,...,None,36,Regular Season,Estadio Abanca-Riazor,Carlos Clos Gómez,Víctor Sánchez del Amo,Juan Eduardo Esnáider Belén,1.1.0,2,2
3,3825855,2016-05-02,20:30:00.000,Spain - La Liga,2015/2016,Málaga,Levante UD,3,1,available,...,None,36,Regular Season,Estadio La Rosaleda,Iñaki Bikandi Garrido,Javier Gracia Carlos,Joan Francesc Ferrer Sicilia,1.1.0,2,2
4,3825908,2016-05-15,19:00:00.000,Spain - La Liga,2015/2016,Espanyol,Eibar,4,2,available,...,None,38,Regular Season,RCDE Stadium,Mario Melero López,Constantin Gâlcă,José Luis Mendilibar Etxebarria,1.1.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,266467,2015-09-23,20:00:00.000,Spain - La Liga,2015/2016,Celta Vigo,Barcelona,4,1,available,...,2021-06-13T16:17:31.694,5,Regular Season,Abanca-Balaídos,Alberto Undiano Mallenco,Manuel Eduardo Berizzo,Luis Enrique Martínez García,1.1.0,2,2
376,267273,2016-02-20,17:00:00.000,Spain - La Liga,2015/2016,Las Palmas,Barcelona,1,2,available,...,2021-06-13T16:17:31.694,25,Regular Season,Estadio de Gran Canaria,NaN,Enrique Setién Solar,Luis Enrique Martínez García,1.1.0,2,2
377,266490,2015-09-20,20:30:00.000,Spain - La Liga,2015/2016,Barcelona,Levante UD,4,1,available,...,2021-06-13T16:17:31.694,4,Regular Season,Spotify Camp Nou,David Fernández Borbalan,Luis Enrique Martínez García,Luis Lucas Alcaraz González,1.1.0,2,2
378,266254,2016-02-14,21:30:00.000,Spain - La Liga,2015/2016,Barcelona,Celta Vigo,6,1,available,...,2021-06-13T16:17:31.694,24,Regular Season,Spotify Camp Nou,NaN,Luis Enrique Martínez García,Manuel Eduardo Berizzo,1.1.0,2,2


In [19]:
our_match_id = 266467

In [20]:
home = matches[matches.match_id==our_match_id].home_team.reset_index()
home_team = home.home_team[0]

In [21]:
lineup = sb.lineups(match_id=our_match_id)


d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [22]:
events = sb.events(match_id=our_match_id)
events = events.sort_values(by=["minute"])

In [23]:
def extract_last_position(position_list):
    if len(position_list) > 0:
        last_position = position_list[-1]['position']
    else:
        last_position = None
    return last_position


In [24]:
home = home_team
home_goal = 0
away_goal = 0
home_list = []
away_list = []
for index, row in events.iterrows():
    if row.shot_outcome =="Goal":
        if row.team ==home:
            home_goal+=1
        else:
            away_goal+=1
    home_list.append(home_goal)
    away_list.append(away_goal)
events["home_score"]=home_list
events["away_score"]=away_list
events["loser"] = np.where(((events.team==home_team) &(events.home_score<events.away_score))|((events.team!=home_team) &(events.home_score>events.away_score)),1,0)
events["Time"] = events.minute.astype("str")+":"+events.second.astype("str")


In [25]:
events[["home_score","away_score","Time","team","shot_outcome","timestamp","loser"]][events.shot_outcome=="Goal"]

,home_score,away_score,Time,team,shot_outcome,timestamp,loser
3250,1,0,25:31,Celta Vigo,Goal,00:25:31.803,0
3251,2,0,29:6,Celta Vigo,Goal,00:29:06.275,0
3262,3,0,55:40,Celta Vigo,Goal,00:10:40.851,0
3273,3,1,79:36,Barcelona,Goal,00:34:36.170,1
3275,4,1,82:5,Celta Vigo,Goal,00:37:05.236,0


In [26]:
results = pd.DataFrame()
for team in events.team.unique():
    df_event = events[(events.team==team) & (events.substitution_outcome=="Tactical")&(events.loser==1)]
    df_lineup = lineup[team]
    df_lineup['last_position'] = df_lineup['positions'].apply(extract_last_position)
    merge_one = df_event[["minute","home_score","away_score","player",'substitution_replacement',"Time","team"]].merge(df_lineup[["player_name","last_position"]], left_on = "player", right_on = "player_name").rename(columns={"last_position":"starter_pos"})
    merge_two = merge_one[["minute","home_score","away_score","Time","team","player_name","starter_pos",'substitution_replacement']].merge(df_lineup[["player_name","last_position"]], right_on = "player_name", left_on = "substitution_replacement").rename(columns={"last_position":"sub_pos","player_name_x":"starter_name","substitution_replacement":"sub_name"}).drop("player_name_y",axis=1)
    merge_two["match_id"] = our_match_id
    results = pd.concat([results,merge_two],axis=0)
results

,minute,home_score,away_score,Time,team,starter_name,starter_pos,sub_name,sub_pos,match_id
0,57,3,0,57:7,Barcelona,Sergi Roberto Carnicer,Right Center Midfield,Munir El Haddadi Mohamed,Right Midfield,266467
1,65,3,0,65:53,Barcelona,Sergio Busquets i Burgos,Right Center Midfield,Ivan Rakitić,Right Center Midfield,266467


In [27]:
results = pd.DataFrame()
xg_share_df = pd.DataFrame()
for match in matches.match_id:
    events = sb.events(match_id = match)
    lineup = sb.lineups(match_id = match)
    home = home_team
    home_goal = 0
    away_goal = 0
    home_list = []
    away_list = []
    for index, row in events.iterrows():
        if row.shot_outcome =="Goal":
            if row.team ==home:
                home_goal+=1
            else:
                away_goal+=1
        home_list.append(home_goal)
        away_list.append(away_goal)
    events["home_score"]=home_list
    events["away_score"]=away_list
    events["loser"] = np.where(((events.team==home_team) &(events.home_score<events.away_score))|((events.team!=home_team) &(events.home_score>events.away_score)),1,0)
    events["Time"] = events.minute.astype("str")+":"+events.second.astype("str")
    for team in events.team.unique():
        df_event = events[(events.team==team) & (events.substitution_outcome=="Tactical")&(events.loser==1)]
        df_lineup = lineup[team]
        df_lineup['last_position'] = df_lineup['positions'].apply(extract_last_position)
        merge_one = df_event[["minute","home_score","away_score","player",'substitution_replacement',"Time","team"]].merge(df_lineup[["player_name","last_position"]], left_on = "player", right_on = "player_name").rename(columns={"last_position":"starter_pos"})
        merge_two = merge_one[["minute","home_score","away_score","Time","team","player_name","starter_pos",'substitution_replacement']].merge(df_lineup[["player_name","last_position"]], right_on = "player_name", left_on = "substitution_replacement").rename(columns={"last_position":"sub_pos","player_name_x":"starter_name","substitution_replacement":"sub_name"}).drop("player_name_y",axis=1)
        merge_two["match_id"] = match
        results = pd.concat([results,merge_two],axis=0)
results

d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
d:\SoccerData\StatsBomb24\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
d:\SoccerD

,minute,home_score,away_score,Time,team,starter_name,starter_pos,sub_name,sub_pos,match_id
0,58,0,2,58:51,Celta Vigo,Daniel Wass,Center Attacking Midfield,Iago Aspas Juncal,Right Wing,3825882
1,70,0,2,70:59,Celta Vigo,Fabián Ariel Orellana Valenzuela,Center Attacking Midfield,Théo Bongonda Mbul''Ofeko Batombo,Left Wing,3825882
2,81,0,2,81:6,Celta Vigo,Pedro Pablo Hernández,Left Defensive Midfield,Nemanja Radoja,Left Defensive Midfield,3825882
0,59,1,0,59:14,Sporting Gijón,Carlos Carmona Bonet,Right Wing,Carlos Castro García,Right Wing,3825832
1,68,1,0,68:10,Sporting Gijón,Pablo Pérez Rodríguez,Center Attacking Midfield,Alen Halilović,Center Attacking Midfield,3825832
...,...,...,...,...,...,...,...,...,...,...
0,57,4,1,57:7,Barcelona,Sergi Roberto Carnicer,Right Center Midfield,Munir El Haddadi Mohamed,Right Midfield,266467
1,65,4,1,65:53,Barcelona,Sergio Busquets i Burgos,Right Center Midfield,Ivan Rakitić,Right Center Midfield,266467
0,64,1,6,64:21,Celta Vigo,Pedro Pablo Hernández,Left Center Midfield,Marcelo Alfonso Díaz Rojas,Left Defensive Midfield,266254
1,76,1,6,76:56,Celta Vigo,John Guidetti,Center Forward,Dejan Dražić,Left Midfield,266254


In [28]:
xg_share_df = pd.DataFrame()
for index, row in results.iterrows():
    sub_time = row["minute"]
    before = events[["home_score","away_score","team","shot_statsbomb_xg"]][(events.minute<sub_time) &(events.minute>=(sub_time-15))].groupby(["team"],as_index=False).agg({"shot_statsbomb_xg":"sum","home_score":"max","away_score":"max"})#/events[["shot_statsbomb_xg"]][(events.minute<sub_time) &(events.minute>=(sub_time-10))].sum()
    before["xg_share"] = before["shot_statsbomb_xg"]/before["shot_statsbomb_xg"].sum()
    after = events[["team","shot_statsbomb_xg"]][(events.minute>=sub_time) &(events.minute<=(sub_time+15))].groupby("team",as_index=False).sum()
    after["xg_share"] = after["shot_statsbomb_xg"]/after["shot_statsbomb_xg"].sum()
    result = before.rename(columns={"xg_share":"before_xg_share"})
    result["Time"]= row.Time
    result = result.merge(after[["team","xg_share"]], on = "team").rename(columns={"xg_share":"after_xg_share"})
    result["xg_change"]=result["after_xg_share"]/result["before_xg_share"]
    xg_share_df = pd.concat([xg_share_df,result], axis = 0)
xg_share_df


,team,shot_statsbomb_xg,home_score,away_score,before_xg_share,Time,after_xg_share,xg_change
0,Barcelona,0.288723,0,2,0.683604,58:51,0.554904,0.811733
1,Valencia,0.133631,0,2,0.316396,58:51,0.445096,1.406768
0,Barcelona,0.116019,0,2,0.554904,70:59,1.000000,1.802112
1,Valencia,0.093060,0,2,0.445096,70:59,0.000000,0.000000
0,Barcelona,0.137484,0,2,1.000000,81:6,0.526327,0.526327
...,...,...,...,...,...,...,...,...
1,Valencia,0.075907,0,2,0.245982,64:21,0.291095,1.183401
0,Barcelona,0.239363,0,2,0.720055,76:56,0.526327,0.730954
1,Valencia,0.093060,0,2,0.279945,76:56,0.473673,1.692021
0,Barcelona,0.137484,0,2,1.000000,82:21,0.526327,0.526327
